In [1]:
import pandas as pd
import sqlite3

In [2]:
db = '../../../data/elections_2019/db.sqlite3'
with sqlite3.connect(db) as con:
    df = pd.read_sql_query("SELECT * FROM candidates_elections_2019", con)

## create "simplified_party" to group some party together

In [3]:
keep = ['N-VA', 'PS', 'MR', 'ECOLO', 'VLAAMS BELANG', 'CD&V', 'Open Vld', 'GROEN', 'CDH', 'DéFI']
change = {
    'PTB': 'PTB & PVDA',
    'PVDA': 'PTB & PVDA',
    'PTB*PVDA': 'PTB & PVDA',
    'sp.a': 'sp.a (+ one.brussels)',
    'sp.a-one.brussels': 'sp.a (+ one.brussels)',
    'one.brussels-sp.a': 'sp.a (+ one.brussels)',
}
def simplify(party):
    if party in keep:
        return party
    if party in change:
        return change[party]
    return 'Autres'
df['simplified_party'] = df.party.apply(simplify)

## Add a list id and list length to each candidate

In [4]:
print(len(df), len(df.drop_duplicates()))
list_sizes = df.groupby(['party', 'status', 'election', 'circonscription']).count()[['position']].rename(columns={'position': 'list_size'}).reset_index()
list_sizes['list_id'] = range(1, len(list_sizes) + 1)
df = df.merge(list_sizes, on=['party', 'status', 'election', 'circonscription'])
print(len(df), len(df.drop_duplicates()))

6927 6927
6927 6927


## Add over- and under-perform indicator

In [5]:
def compute_performance(l):
    not_last = l.loc[l['position'] != l['list_size'], ['candidate_id', 'position', 'votes']]
    not_last['ranking'] = not_last['votes'].rank(ascending=False)
    not_last['performance'] = (not_last.position - not_last.ranking)
    not_last.performance = not_last.performance.apply(lambda x: "better" if x > 0 else ("worse" if x < 0 else "equal"))

    l = l.merge(not_last[['candidate_id', 'performance']], on='candidate_id', how='left')
    l.loc[l['position'] == l['list_size'], 'performance'] = 'last'

    return l

df = df.groupby('list_id').apply(compute_performance).reset_index(drop=True).sort_values('candidate_id')


## Export

In [6]:
df.to_json('../data/candidates.json', orient='records')
df.to_csv('../data/candidates.csv', index=False)

# with open('../js/data.js', 'w+') as f:
#     f.write('candidates = ')
#     f.write(df.to_json(orient='records'))
#     f.write(';')

In [9]:
df.loc[df.status == 'effectif', ['party', 'elu']].groupby('party').sum().sort_values(by='elu', ascending=False)

,elu
party,
N-VA,66
PS,62
MR,49
ECOLO,45
VLAAMS BELANG,45
CD&V,34
Open Vld,33
PTB,28
GROEN,27
